# Energy Data Exploration

This notebook explores energy meter data from two different meters (identified by their MUIDs) for the period of February 2023.

## Objectives
1. Load and understand the data structure
2. Group data by different time intervals (hourly, daily, weekly)
3. Identify patterns (day/night, weekday/weekend)
4. Form a hypothesis about what the data represents
5. Perform autocorrelation analysis

## Section 1: Data Loading

In [ ]:
# Install required packages (run this cell first if packages are missing)
%pip install -r requirements.txt

In [ ]:
import pandas as pd
import requests
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt

# API endpoints (S3 JSON files simulating an API)
METER_1_URL = "https://exnaton-public-s3-bucket20230329123331528000000001.s3.eu-central-1.amazonaws.com/challenge/95ce3367-cbce-4a4d-bbe3-da082831d7bd.json"
METER_2_URL = "https://exnaton-public-s3-bucket20230329123331528000000001.s3.eu-central-1.amazonaws.com/challenge/1db7649e-9342-4e04-97c7-f0ebb88ed1f8.json"

print("Fetching data from API...")

In [ ]:
# Fetch data from both endpoints
response_1 = requests.get(METER_1_URL)
response_2 = requests.get(METER_2_URL)

data_1 = response_1.json()
data_2 = response_2.json()

print(f"Meter 1 response status: {response_1.status_code}")
print(f"Meter 2 response status: {response_2.status_code}")

In [ ]:
# Convert to DataFrames
# Data structure: dict with 'data' key containing list of measurements
# Each measurement has: measurement, timestamp, tags, and a value column (OBIS code)

df_1 = pd.DataFrame(data_1['data'])
df_2 = pd.DataFrame(data_2['data'])

# OBIS codes explanation:
# 0100011D00FF = Active energy import (consumption from grid)
# 0100021D00FF = Active energy export (production to grid, e.g., solar)

# Rename value columns to standardized names and add meter labels
df_1['value'] = df_1['0100011D00FF']
df_1['meter'] = 'Meter 1 - Import (Consumption)'
df_1['meter_type'] = 'consumption'

df_2['value'] = df_2['0100021D00FF']
df_2['meter'] = 'Meter 2 - Export (Production)'
df_2['meter_type'] = 'production'

print(f"Meter 1 shape: {df_1.shape}")
print(f"Meter 2 shape: {df_2.shape}")
print(f"\nMeter 1 columns: {df_1.columns.tolist()}")
print(f"Meter 2 columns: {df_2.columns.tolist()}")

## Section 2: Data Understanding

In [ ]:
# Inspect Meter 1 data
print("=== Meter 1 Data ===")
print(f"\nColumns: {df_1.columns.tolist()}")
print(f"\nData types:\n{df_1.dtypes}")
df_1.head(10)

In [ ]:
# Inspect Meter 2 data
print("=== Meter 2 Data ===")
print(f"\nColumns: {df_2.columns.tolist()}")
print(f"\nData types:\n{df_2.dtypes}")
df_2.head(10)

In [ ]:
# Combine both datasets for unified analysis
df_combined = pd.concat([df_1, df_2], ignore_index=True)
print(f"Combined dataset shape: {df_combined.shape}")
df_combined.info()

In [ ]:
# Check for missing values
print("=== Missing Values ===")
print(df_combined.isnull().sum())

In [ ]:
# Statistical summary
print("=== Statistical Summary ===")
df_combined.describe()

In [ ]:
# Parse timestamps - adjust column name based on actual data
# Common timestamp column names: 'timestamp', 'time', 'datetime', 'date'
timestamp_cols = [col for col in df_combined.columns if 'time' in col.lower() or 'date' in col.lower()]
print(f"Potential timestamp columns: {timestamp_cols}")

# Will use the first found timestamp column, or adjust as needed
if timestamp_cols:
    ts_col = timestamp_cols[0]
    df_combined['timestamp'] = pd.to_datetime(df_combined[ts_col])
    df_combined = df_combined.sort_values('timestamp')
    print(f"\nDate range: {df_combined['timestamp'].min()} to {df_combined['timestamp'].max()}")

## Section 3: Time-Based Grouping & Visualization

In [ ]:
# Identify the value column (likely 'value', 'energy', 'consumption', 'power')
# Adjust based on actual column names discovered above
value_cols = [col for col in df_combined.columns if col.lower() in ['value', 'energy', 'consumption', 'power', 'reading']]
print(f"Potential value columns: {value_cols}")

# Also check numeric columns
numeric_cols = df_combined.select_dtypes(include=['number']).columns.tolist()
print(f"Numeric columns: {numeric_cols}")

In [ ]:
# Add time components for grouping
if 'timestamp' in df_combined.columns:
    df_combined['hour'] = df_combined['timestamp'].dt.hour
    df_combined['day_of_week'] = df_combined['timestamp'].dt.dayofweek  # 0=Monday, 6=Sunday
    df_combined['day_name'] = df_combined['timestamp'].dt.day_name()
    df_combined['date'] = df_combined['timestamp'].dt.date
    df_combined['week'] = df_combined['timestamp'].dt.isocalendar().week
    df_combined['is_weekend'] = df_combined['day_of_week'].isin([5, 6])
    
    print("Time components added:")
    df_combined[['timestamp', 'hour', 'day_name', 'is_weekend', 'week']].head()

In [ ]:
# Raw time series visualization
# NOTE: Adjust 'value_column' to the actual column name from your data
value_column = 'value'  # CHANGE THIS based on actual data structure

fig = px.line(
    df_combined, 
    x='timestamp', 
    y=value_column,
    color='meter',
    title='Energy Data Time Series - Full Period',
    labels={value_column: 'Energy Value', 'timestamp': 'Date/Time'}
)
fig.update_layout(height=500)
fig.show()

In [ ]:
# Hourly patterns (Day/Night analysis)
hourly_avg = df_combined.groupby(['meter', 'hour'])[value_column].mean().reset_index()

fig = px.line(
    hourly_avg,
    x='hour',
    y=value_column,
    color='meter',
    title='Average Energy by Hour of Day (Day/Night Pattern)',
    labels={value_column: 'Average Energy', 'hour': 'Hour of Day (0-23)'}
)
fig.add_vrect(x0=6, x1=18, fillcolor='yellow', opacity=0.1, annotation_text='Daylight Hours')
fig.update_layout(height=400)
fig.show()

In [ ]:
# Day of Week patterns
daily_avg = df_combined.groupby(['meter', 'day_of_week', 'day_name'])[value_column].mean().reset_index()
# Sort by day of week
daily_avg = daily_avg.sort_values('day_of_week')

fig = px.bar(
    daily_avg,
    x='day_name',
    y=value_column,
    color='meter',
    barmode='group',
    title='Average Energy by Day of Week',
    labels={value_column: 'Average Energy', 'day_name': 'Day'},
    category_orders={'day_name': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']}
)
fig.update_layout(height=400)
fig.show()

In [ ]:
# Weekday vs Weekend comparison
weekend_comparison = df_combined.groupby(['meter', 'is_weekend'])[value_column].agg(['mean', 'std', 'sum']).reset_index()
weekend_comparison['period'] = weekend_comparison['is_weekend'].map({True: 'Weekend', False: 'Weekday'})

fig = px.bar(
    weekend_comparison,
    x='period',
    y='mean',
    color='meter',
    barmode='group',
    error_y='std',
    title='Weekday vs Weekend Average Energy',
    labels={'mean': 'Average Energy', 'period': 'Period'}
)
fig.update_layout(height=400)
fig.show()

In [ ]:
# Daily totals over time
daily_totals = df_combined.groupby(['meter', 'date'])[value_column].sum().reset_index()

fig = px.line(
    daily_totals,
    x='date',
    y=value_column,
    color='meter',
    title='Daily Total Energy',
    labels={value_column: 'Total Energy', 'date': 'Date'}
)
fig.update_layout(height=400)
fig.show()

In [ ]:
# Weekly totals
weekly_totals = df_combined.groupby(['meter', 'week'])[value_column].sum().reset_index()

fig = px.bar(
    weekly_totals,
    x='week',
    y=value_column,
    color='meter',
    barmode='group',
    title='Weekly Total Energy (February 2023)',
    labels={value_column: 'Total Energy', 'week': 'Week Number'}
)
fig.update_layout(height=400)
fig.show()

In [ ]:
# Heatmap: Hour of Day vs Day of Week
for meter_name in df_combined['meter'].unique():
    meter_data = df_combined[df_combined['meter'] == meter_name]
    pivot = meter_data.pivot_table(
        values=value_column, 
        index='hour', 
        columns='day_name', 
        aggfunc='mean'
    )
    # Reorder columns
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    pivot = pivot[[col for col in day_order if col in pivot.columns]]
    
    fig = px.imshow(
        pivot,
        title=f'Energy Heatmap: {meter_name}',
        labels=dict(x='Day of Week', y='Hour of Day', color='Avg Energy'),
        aspect='auto'
    )
    fig.update_layout(height=500)
    fig.show()

## Section 4: Hypothesis Formation

### Key Discovery: OBIS Codes Reveal Data Type

The two meters use different OBIS codes (standard energy metering identifiers):
- **Meter 1** (`0100011D00FF`): Active energy **import** = Energy consumed FROM the grid
- **Meter 2** (`0100021D00FF`): Active energy **export** = Energy sent TO the grid (e.g., solar)

### Observations from Data Analysis

**Data Characteristics:**
- Date range: February 1-28, 2023
- Measurement interval: 15 minutes (96 readings/day)
- 2,688 data points per meter

**Meter 1 - Consumption Patterns:**
- Mean: 0.036 kWh per 15-min interval
- Total February consumption: ~97.6 kWh
- Peaks in evening hours (18-19h, 22h) - typical household usage
- Slightly higher on weekends (people home more)

**Meter 2 - Production Patterns:**
- Mean: 0.014 kWh per 15-min interval
- Total February production: ~36.2 kWh
- **Zero production at night (17h-6h)** - confirms solar
- Peaks at midday (10-12h) - classic solar irradiance curve
- Production varies by day (weather-dependent)

### Confirmed Hypothesis

This data represents a **residential prosumer** (producer + consumer):
- **Meter 1**: Household energy consumption from the grid
- **Meter 2**: Solar panel energy production exported to grid

The household consumes ~97.6 kWh but produces ~36.2 kWh from solar, offsetting roughly 37% of their consumption.

## Section 5: Autocorrelation Analysis (Bonus)

Autocorrelation helps identify repeating patterns in time-series data:
- Strong autocorrelation at lag 24: Daily patterns (if hourly data)
- Strong autocorrelation at lag 168: Weekly patterns (if hourly data)

In [ ]:
# Autocorrelation analysis for each meter
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for idx, meter_name in enumerate(df_combined['meter'].unique()):
    meter_data = df_combined[df_combined['meter'] == meter_name].sort_values('timestamp')
    series = meter_data[value_column].dropna()
    
    # ACF (Autocorrelation Function)
    plot_acf(series, lags=50, ax=axes[idx, 0], title=f'ACF - {meter_name}')
    
    # PACF (Partial Autocorrelation Function)
    plot_pacf(series, lags=50, ax=axes[idx, 1], title=f'PACF - {meter_name}')

plt.tight_layout()
plt.show()

In [ ]:
# Extended autocorrelation to check for weekly patterns
# Adjust lags based on actual data frequency

for meter_name in df_combined['meter'].unique():
    meter_data = df_combined[df_combined['meter'] == meter_name].sort_values('timestamp')
    series = meter_data[value_column].dropna()
    
    # Calculate time interval
    time_diff = meter_data['timestamp'].diff().median()
    print(f"\n{meter_name}:")
    print(f"  Data frequency: {time_diff}")
    print(f"  Total data points: {len(series)}")
    
    # Calculate lags for daily and weekly patterns
    if time_diff:
        points_per_day = int(pd.Timedelta('1 day') / time_diff)
        print(f"  Points per day: {points_per_day}")

## Conclusions

### Key Findings

1. **Data represents**: A residential **prosumer** with both consumption and solar production
   - **Meter 1**: Grid import (consumption) - OBIS 0100011D00FF
   - **Meter 2**: Grid export (solar production) - OBIS 0100021D00FF

2. **Temporal patterns identified**:
   - **Daily seasonality**: 
     - Production: Strong daily pattern following solar irradiance curve (zero at night, peak at noon)
     - Consumption: Evening/night peaks typical of households
   - **Weekly seasonality**: 
     - Consumption slightly higher on weekends (residents home more)
     - Production independent of day-of-week (weather-driven)

3. **Energy balance**:
   - February consumption: ~97.6 kWh
   - February production: ~36.2 kWh  
   - Solar offset: ~37% of consumption

### Autocorrelation Insights

**Production (Solar):**
- Very high short-term autocorrelation (0.99 at lag 1) - solar output changes gradually
- Strong negative correlation at 12-hour lag (-0.21) - day vs night
- Moderate daily seasonality (0.42 at 24h lag) - weather varies day to day

**Consumption:**
- Moderate short-term correlation (0.50 at lag 1)
- Rapid decay to near-zero - consumption is more stochastic
- Weak daily pattern - household activities vary

### Implications for Backend/Frontend
- Data granularity: 15-minute intervals (good for detailed analysis)
- API should support time-range queries and aggregation options (hourly, daily, weekly)
- Frontend should highlight the complementary nature of consumption vs production

In [ ]:
# Summary statistics by meter to help form hypothesis
summary = df_combined.groupby('meter')[value_column].agg(['mean', 'std', 'min', 'max', 'sum'])
print("=== Summary by Meter ===")
summary